# ApertureDB

[ApertureDB](https://docs.aperturedata.io) is a database that stores, indexes, and manages multi-modal data like text, images, videos, bounding boxes, and embeddings, together with their associated metadata.

This notebook explains how to use the embeddings functionality of ApertureDB.

To run, you should have an [ApertureDB instance up and running](https://docs.aperturedata.io/HowToGuides/start/Setup).

In [1]:
%pip install --upgrade --quiet  aperturedb

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip freeze | grep langchain

langchain==0.1.16
langchain-community==0.0.32
langchain-core==0.1.42
langchain-text-splitters==0.0.1
Note: you may need to restart the kernel to use updated packages.


We shall be using aperturedb Vectorstore to index embeddings from a URL.

We want to use OllamaEmbeddings so we have to import the necessary modules.

Ollama can be set up as a docker container as per the [official guide](https://hub.docker.com/r/ollama/ollama)

In [4]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

#For loading documents from web
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.aperturedata.io")
docs = loader.load()

from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

from langchain_community.vectorstores import ApertureDB
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
embeddings.embed_query("What is the capital of France?")

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f66468a76d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [5]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector_db = ApertureDB.from_documents(documents, embeddings)

llm = Ollama(model="llama2")

AssertionError: Dimensions must be set for new descriptorsets

In [7]:
llm = Ollama(model="llama2")
llm.embed_query("What is the capital of France?")

AttributeError: 'Ollama' object has no attribute 'embed_query'

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


from langchain.chains import create_retrieval_chain

retriever = vector_db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can aperturedata store images?"})
print(response["answer"])

NameError: name 'llm' is not defined